# SAM

Segment Anything és un projecte de **Meta** amb dues grans aportacions:

- Un gran conjunt de dades per a la segmentació d'imatges
- El model Segment Anything (SAM) com a model de base per a la segmentació d'imatges

Va ser introduït a l'article **Segment Anything** per Alexander Kirillov _et al._** (Abril 2023) [enllaç](https://arxiv.org/pdf/2304.02643.pdf)

Aquest model s'inspira en el camp de la NLP (_Natural Language Processing_), on la creació de models base (_foundation models_) i els grans conjunts de dades (per valor de milers de milions de dades) s'han convertit en la manera habitual de fer feina.


Com ja sabem, la segmentació d'imatges té diversos usos, aquests inclouen: l'anàlisi d'imatges biomèdiques, l'edició de fotografies i la conducció autònoma, entre d'altres. Per resoldre qualsevol d'aquests problemes, cal entrenar models especialitzats per cada una de les tasques que hem citat (és més, per cada un dels subproblemes que es deriven de cada tasca i per cada cas en particular). Això requereix un ampli coneixement del domini del problema i el temps necessari per a la recollida de dades específiques, per no parlar de les hores d'entrenament i ajustamen que són necessàries per als models d'aprenentatge profund.

Enllaços:
- [Repositòri Oficial](https://github.com/facebookresearch/segment-anything)
- [Demo for dummies](https://segment-anything.com/demo)


## Model

SAM és un model d'aprenentatge profund (basat en _transformers_). I com amb qualsevol aprenentatge profund, s'ha entrenat en un gran nombre d'imatges i màscares: **més de mil milions de màscares en 11 milions d'imatges**.

Com SAM és una model base, i està preparat per segmentar qualsevol tipus d'imatge, pot rebre indicacions dels usuaris sobre quina àrea segmentar. Actualment podem proporcionar tres indicacions diferents a SAM:

- Indicar punts que contenen i no contenen el que volem segmentar.
- Dibuixant un quadre delimitador (_bounding box_).
- Dibuixant una màscara genearl sobre un objecte.

En l'article és parla de preparar un _promt_ per poder especificar el que es vol segmentar emprant text.

L'arquitectura de la xarxa és la següent:

![SAM](https://learnopencv.com/wp-content/uploads/2023/04/segment-anything-model.png)


Les caracterìstiques més importants del model (secció 3 de l'article) són:

**Encoder**. Vision Transformer (ViT) basat en la idea d'un MAE (Masked AutoEncoder) El codificador d'imatge s'executa una vegada per imatge i es pot aplicar abans de sol·licitar el
_prompt_ al model.

**Codificador de _prompts_**. S'en consideren dos tipus: dispersos (punts, _bounding boxes_, text) i densos (màscares). Representen els punts i les _bounding boxes_ mitjançant codificacions posicionals amb altres informacions apreses per a cada tipus d'_input_ i text de forma lliure amb un codificador de text anomenat CLIP. Les indicacions denses (és a dir, les màscares) s'incorporen mitjançant l'ús convolucions i es sumen (concatenen) amb la codificació obtinguda de l'encoder de la imatge.

**Descodificador de màscares**. El descodificador de màscara mapeja de manera eficient els _embeddings_ d'una imatge i el resultat del codificador de _prompts_ utilitzant una modificació d'un bloc descodificador Transformer 

**Resolució de l'ambigüitat**. Amb una sortida, el model farà una mitjana de diverses màscares vàlides si se li dóna una indicació ambigua. Per solucionar-ho, modifiquem el model per predir múltiples màscares de sortida per a un sol _prompt_ d'entreada. Experimentalment s'ha arribat a la conclusió que 3 màscares de sortida de màscara són suficients per abordar els casos més habituals (les màscares imbricades solen tenir tres profunditats com a màxim: senceres, parcials i subparts). 

**Eficiència**. El disseny global del model està motivat en gran mesura per l'eficiència. Donat un _embedding_ precalculat d'una imatge, el codificador de _promts_ i el descodificador de màscares s'executen en un navegador web, a la CPU, en ∼50 ms. Aquest rendiment en temps d'execució permet una indicació interactiva en temps real del model.

**Pèrdues i entrenament.** Es supervisa la predicció de les màscares amb la combinació lineal d'una funció de pèrdua focal [enllaç](https://paperswithcode.com/method/focal-loss) i una funció de pèrdua de dice (ja emprada al model YOLO). L'entrenament es realitza utilitzant una barreja d'indicacions geomètriques, es simula un entrenament interactiu mitjançant un mostreig aleatori de prompts en 11 rondes per màscara.


## Dades

Com ja sabem, la base de qualsevol model d'aprenentatge profund innovador és el conjunt de dades en què s'ha entrenat. El conjunt de dades de SAM conté més d'**11 milions d'imatges i 1.100 milions de màscares**. El conjunt de dades final s'anomena conjunt de dades SA-1B.

Segurament es necessita aquest conjunt de dades per entrenar un model de capacitat de Segment Anything. Però també sabem que aquests conjunts de dades no existeixen i que és impossible anotar manualment tantes imatges.Per tant es necessitar l'ajut de SAM per anotar el conjunt de dades: Els anotadors de dades van utilitzar SAM per anotar imatges de manera interactiva i les dades anotades es van utilitzar per entrenar SAM. Aquest procés es va repetir, cosa que va donar lloc al motor de dades en bucle de SAM.

Aquest motor de dades + formació del SAM al conjunt de dades té tres etapes:

- Etapa Manual Assistida
- Etapa semiautomàtica
- Etapa totalment automàtica

En la primera etapa, els anotadors van utilitzar un model SAM prèviament entrenat per segmentar objectes de manera interactiva en imatges al navegador. Els _embeddings_ de les imatges es van calcular prèviament per fer que el procés d'anotació fos fluid i en temps real. Després de la primera etapa, el conjunt de dades constava de 4,3 milions de màscares a partir  de 120.000 imatges. El model Segment Anything es va tornar a entrenar en aquest conjunt de dades.

En la segona etapa semiautomàtica, els objectes destacats ja estaven segmentats mitjançant SAM. Els anotadors també van anotar objectes menys destacats que no tenien anotació. Aquesta etapa va donar lloc a 5,9 milions de màscares addicionals en 180.000 imatges en les quals es va tornar a entrenar SAM.

A l'etapa "totalment automàtica" final, l'anotació la va fer íntegrament SAM. En aquesta etapa, ja s'havia entrenat el model en més de 10 milions de màscares que ho van fer possible. La generació automàtica de màscares es va aplicar 11M a imatges , donant lloc a 1,1B  màscares.
En paraules del seus creadors:

"La versió final del conjunt de dades Segment Anything el converteix en el conjunt de dades de segmentació d'imatges més gran disponible públicament. En comparació amb OpenImages V5, hi ha 6 vegades més imatges i 400 vegades més màscares al conjunt de dades."

## Tutorial

Hi ha 3 models diferents de SAM:

- ViT-B SAM
- ViT-L SAM
- ViT-H SAM

Com ja vàrem veure l'altra dia amb YOLO, el projecte també té paquet que podem instal·lar l mitjançant l'ordre següent:

In [ ]:
#!pip install git+https://github.com/facebookresearch/segment-anything.git

Nosaltres utilitzarem els pesos i el model SAM oficials per executar inferències en algunes imatges. Podeu trobar tots els pesos oficials del model al repositori oficial [enllaç](https://github.com/facebookresearch/segment-anything#model-checkpoints).

Anem a veure un exemple dús del model:

In [ ]:
#!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import torch

from segment_anything import SamAutomaticMaskGenerator, sam_model_registry


A continuació teniu algunes funcions de suport per dibuixar i ajudar a comprendre el resultat. Aquestes funcions són de la documentació oficial [aquí](https://github.com/facebookresearch/segment-anything/blob/main/notebooks/predictor_example.ipynb)

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        np.dstack((img, m*0.35))
        ax.imshow(np.dstack((img, m*0.35)))
        
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))    


In [ ]:
use_cuda = True
torch.manual_seed(33)

if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

Anem a predir totes les màscares d'una imatge:

In [ ]:
image_path = "tony_gaga.jpg"

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 9))
plt.imshow(image);

In [ ]:
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth") # Si teniu capacitat de càlcul provau amb el model vit_h
sam.to(device)
mask_generator = SamAutomaticMaskGenerator(sam)  # genera totes les màscares
 

masks = mask_generator.generate(image)
plt.figure(figsize=(12, 9))
plt.imshow(image)
plt.axis('off')
show_anns(masks)

Ara emprarem el _prompt_ per indicar un objecte a segmentar. L'estructura bàsica és la següent:

```
sam = sam_model_registry["<model_type>"](checkpoint="<path/to/checkpoint>")
predictor = SamPredictor(sam)
predictor.set_image(<your_image>)
masks, _, _ = predictor.predict(<input_prompts>)
```

In [ ]:
from segment_anything import SamPredictor, sam_model_registry # necessitam importar el predictor

In [ ]:
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth")
sam.to(device)
predictor = SamPredictor(sam) # genera les màscares que li demanam
predictor.set_image(image)


Per seleccionar un objecte hem de triar un punt. Els punts s'introdueixen al model en format (x,y) i vénen amb les etiquetes 1 (punt d'un objecte) o 0 (punt de fons). Es poden introduir diversos punts; aquí només en fem servir un. El punt escollit es mostrarà com una estrella a la imatge.

In [ ]:
input_point = np.array([[500, 500]])
input_label = np.array([1])

plt.figure(figsize=(10,10))
plt.imshow(image)
show_points(input_point, input_label, plt.gca())
plt.axis('on')
plt.show()  

Farem la predicciió amb `SamPredictor.predict`. El model retorna màscares, prediccions de qualitat per a aquestes màscares i _logits_  (prediccions) de màscares de baixa resolució que es poden passar a la següent iteració de predicció.

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True)


In [ ]:
masks.shape

In [ ]:
for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    show_mask(mask, plt.gca())
    show_points(input_point, input_label, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show()  



### Feina a fer

Seguir el tutorial [aquí](https://github.com/facebookresearch/segment-anything/blob/main/notebooks/predictor_example.ipynb) per poder fer prediccions emprant _bounding boxes_


**Extra 1**

També podeu trobar el model i  altres fitxers de pesos en el repositori de huggingface ([enllaç](https://github.com/facebookresearch/segment-anything#model-checkpoints)) que a més ens aporta un conjunt de funcions i llibreries adicionals per tasques d'aprenentatge profund.

**Extra 2**

[Tutorial Roboflow](https://blog.roboflow.com/how-to-use-segment-anything-model-sam/)

_In this written tutorial (and the video below), we will explore how to use SAM to generate masks automatically, create segmentation masks using bounding boxes, and convert object detection datasets into segmentation masks. If you're interested in using SAM to label data for computer vision, Roboflow Annotate uses SAM to power automated polygon labeling in the browser which you can try for free._
